In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from sklearn.metrics import r2_score

# Data preparation

In [2]:
data = pd.read_csv('SNP_gwas_mc_merge_nogc.tbl.uniq.txt', sep="\t")
data.head(5)

,SNP,A1,A2,Freq1.Hapmap,b,se,p,N
0,rs1000000,G,A,0.6333,0.0001,0.0044,0.98190,231410.0
1,rs10000010,T,C,0.5750,-0.0029,0.0030,0.33740,322079.0
2,rs10000012,G,C,0.1917,-0.0095,0.0054,0.07853,233933.0
3,rs10000013,A,C,0.8333,-0.0095,0.0044,0.03084,233886.0
4,rs10000017,C,T,0.7667,-0.0034,0.0046,0.45980,233146.0


In [ ]:
for data.SNP in data: SE_table_1, p_table_1, beta_table_1, N_table_1 = data.se, data.p, data.b, data.N

# Functions to calculate beta, SE and p-value

We are using formula: $T^{2} = \frac{\hat{\beta^{2}}}{Var}$, where $Var = \sigma ^{2}$ 

Standard error given as follows: $SE = \frac{\sigma}{\sqrt{n}} $

In [ ]:
def calc_abs_beta(SE, p, df):
  '''Calculates beta absolute vale given p-value, standard error and df '''
  '''Takes two vectors SE and p, one scalar df'''
  '''Returns vector beta'''
  chi_sq = ss.distributions.chi2.isf(p, df)
  beta_sq = chi_sq * (SE ) ** 2
  return np.sqrt(beta_sq)

In [ ]:
def calc_SE(b, p, df):
  '''Calculates standard error given p-value, absolute value of beta and df '''
  '''Takes two vectors b and p, one scalar df'''
  '''Returns vector SE'''
  chi_sq = ss.distributions.chi2.isf(p, df)
  SE_sq = b **2 / chi_sq 
  return np.sqrt(SE_sq )

In [ ]:
def calc_p(b, SE, df):
  '''Calculates standard error given p-value, absolute value of beta and df '''
  '''Takes two vectors b and p, one scalar df'''
  '''Returns vector of p values'''
  chi_sq = b / SE ** 2
  p_val = ss.distributions.chi2.sf(chi_sq, df)
  return 1 - p_val

# Testing function on given data and comparing to initial values from table

Starting from calculating beta, while having values of SE and p-value. We compare absolute values of beta given from table - "From table" and calculated by formula - "Calculated".



In [ ]:
# creating an array of calculated betas
beta_calc = calc_abs_beta(SE_table_1, p_table_1, 1)

Create a table to compare values

In [ ]:
dict_ =  {'Calculated': beta_calc, 'From table': abs(data.b)}
df = pd.DataFrame(data=dict_)
df.head(10)

,Calculated,From table
0,0.000100,0.0001
1,0.002878,0.0029
2,0.009500,0.0095
3,0.009500,0.0095
4,0.003400,0.0034
5,0.002400,0.0024
6,0.000500,0.0005
7,0.033001,0.0330
8,0.010900,0.0109
9,0.000700,0.0007


In [ ]:
# calculate corr coefficient for these values
r2_score(df['Calculated'], df['From table'])

0.9999966436891115

We can see that formula works well enough

Now calculate SE given values of SE and p-value. We compare values of SE given from table - "From table" and calculated by formula - "Calculated".

In [ ]:
# creating an array of calculated betas
SE_calc = calc_SE(beta_table_1, p_table_1, 1)

In [ ]:
dict_2 =  {'Calculated': SE_calc, 'From table': data.se)}
df = pd.DataFrame(data=dict_2)
df.head(10)

,Calculated,From table
0,0.004408,0.0044
1,0.003023,0.0030
2,0.005400,0.0054
3,0.004400,0.0044
4,0.004600,0.0046
5,0.003800,0.0038
6,0.008006,0.0080
7,0.024100,0.0241
8,0.003700,0.0037
9,0.005402,0.0054


We can see that formula works well enough

# Threshold for p-value

Counting the values of SNP with p-value less than $5\cdot 10^{-8}$. We consider them statistically significant 

In [3]:
data[data.p < 5 * 10 **(-8)]['SNP'].count()

1860